In [10]:
from sage.graphs.connectivity import connected_components
from sage.graphs.connectivity import connected_components_number

#m x n mreža, a št. izbrisanih vozlov, b št.izrbirsanih povezav
def mreza(m,n,a,b):
    mreza = graphs.Grid2dGraph(m,n)
    if a > mreza.order():
        print("Za ukaz je na voljo premalo vozlov.")
    else:
        i = 0
        while i < a:
            mreza.delete_vertex(mreza.random_vertex())
            i = i+1
        i = 0
    if b > mreza.size():
        print("Za ukaz je na voljo premalo povezav.")
    else:
        while i < b:
            mreza.delete_edge(mreza.random_edge())
            i = i+1
    return mreza

#reši CLP problem in vrne k
def najkrajsa_razdalja(G, st_centrov):
    K = st_centrov
    razdalje = G.distance_all_pairs()

    p = MixedIntegerLinearProgram(maximization=False)
    x = p.new_variable(binary=True) #x_uv = 1 če mesto u spada k skladišču v (mestu v s skladiščem)
    y = p.new_variable(binary=True) # y_v = 1 če je v mestu v skladišče

    p.set_objective(p['R']) # največja razdalja je spremenljivka

    for u in G:
        p.add_constraint(sum(x[u, v] for v in G) == 1) #za vsako mesto u bo veljalo, da spada pod neko območje mesta v s skladiščem

    p.add_constraint(sum(y[v] for v in G) == K) #vsota skladišč je enaka K

    for u in G:
        for v in G:
            p.add_constraint(x[u, v] <= y[v]) #ne sme se zgoditi, da mesto u pade v območje mesta v, v mestu v pa sploh ni skladišča

    for u in G:
        for v in G:
            if v in razdalje[u]:
                p.add_constraint(razdalje[u][v] * x[u, v] <= p['R']) # če sta vozlišči v isti povezani komponenti, potem omejimo največjo razdaljo                                                                        do skladišča
            else:
                p.add_constraint(x[u, v] == 0) # sicer mesto u ne more pripadati skladišču v
    max_razdalja = p.solve()
    skladisca = [k for k, v in p.get_values(y).items() if v == 1]
    #print(skladisca)
    return max_razdalja


#G = mreza(5,3,3,2)
#slika=G.show()
#najkrajsa_razdalja(G,5)

#print(connected_components(G)) #večdelni graf zapiše po ločenih delih
#print(connected_components_number(G)) #št. delov

In [2]:
#kako se optimalna vrednost spreminja glede na k
def opt_vrednost_k(m,n,a,b,k):
    G = mreza(m,n,a,b)
    seznam_vrednosti = []
    stevilo_komponent = connected_components_number(G)
    for i in range(stevilo_komponent,k+1): ## težave pri določanju najmanjšega i???
        razdalja = round(najkrajsa_razdalja(G, i)) #round za zaokrževanje števil
        seznam_vrednosti.append((razdalja))
    return seznam_vrednosti

#vrne optimalno vrednost R za več ponovitev
#FIKSNO: velikost mreže, število izbrisanih povezav in vozlišč
#SPREMINJAVA: k
def opt_vrednost_za_vec_ponovitev(m,n,a,b,max_stevilo_centrov,stevilo_ponovitev):
    seznam = []
    for i in range(0, stevilo_ponovitev):
        #G = mreza(m,n,a,b)
        razdalje = opt_vrednost_k(m,n,a,b,max_stevilo_centrov)
        seznam.append(razdalje)

    for i in range(len(seznam)):
        while len(seznam[i]) < max_stevilo_centrov:
            seznam[i] = [None] + seznam[i]

    povprecja = []
    for j in range(max_stevilo_centrov):
        vsota = 0
        stevec = 0
        for i in range(stevilo_ponovitev):
            if seznam[i][j] != None:
                vsota += seznam[i][j]
                stevec += 1
        if stevec == 0:
            povprecja.append(None)
        else:
            povprecja.append(vsota/stevec)

    return seznam, povprecja

opt_vrednost_za_vec_ponovitev(5,5,3,2,7,5)

([[5.0, 4.0, 2.0, 2.0, 2.0, 2.0, 1.0],
  [4.0, 3.0, 2.0, 2.0, 2.0, 2.0, 1.0],
  [None, 4.0, 3.0, 2.0, 2.0, 2.0, 1.0],
  [5.0, 3.0, 3.0, 2.0, 2.0, 2.0, 1.0],
  [6.0, 3.0, 3.0, 2.0, 2.0, 2.0, 1.0]],
 [5.0, 3.4, 2.6, 2.0, 2.0, 2.0, 1.0])

In [31]:
import time

#čas izvajanja v odvisnosti od k
def cas_izvajanja_k(m,n,a,b,k):
    G = mreza(m,n,a,b)
    seznam_casov = []
    stevilo_komponent = connected_components_number(G)
    for i in range(stevilo_komponent,k+1):
        zacetni = time.time()
        najkrajsa_razdalja(G,i)
        koncni = time.time() - zacetni
        seznam_casov.append((koncni))
    return seznam_casov

cas_izvajanja_k(3,3,1,1,5)

#vrne povprečne čase(od n ponovitev) za različne k
#FIKSNO: velikost mreže, št. izbrisanih povezav in vozlišč
#SPREMINJAMO: k
def cas_izvajanja_za_vec_ponovitev(m,n,a,b,max_stevilo_centrov,stevilo_ponovitev):
    seznam = []
    for i in range(0, stevilo_ponovitev):
        #G = mreza(m,n,a,b)
        casi = cas_izvajanja_k(m,n,a,b,max_stevilo_centrov)
        seznam.append(casi)

    for i in range(len(seznam)):
        while len(seznam[i]) < max_stevilo_centrov:
            seznam[i] = [None] + seznam[i]

    povprecja = []
    for j in range(max_stevilo_centrov):
        vsota = 0
        stevec = 0
        for i in range(stevilo_ponovitev):
            if seznam[i][j] != None:
                vsota += seznam[i][j]
                stevec += 1
        if stevec == 0:
            povprecja.append(None)
        else:
            povprecja.append(vsota/stevec)

    return seznam, povprecja

cas_izvajanja_za_vec_ponovitev(4,4,5,2,7,5)

([[None,
   None,
   None,
   0.01146841049194336,
   0.014780282974243164,
   0.016327619552612305,
   0.015952110290527344],
  [0.01682591438293457,
   0.051138877868652344,
   0.06979632377624512,
   0.08555340766906738,
   0.09345316886901855,
   0.08623027801513672,
   0.08849048614501953],
  [None,
   None,
   0.011798381805419922,
   0.014562845230102539,
   0.01585078239440918,
   0.021587610244750977,
   0.021541118621826172],
  [None,
   0.013257503509521484,
   0.02094125747680664,
   0.039755821228027344,
   0.04304146766662598,
   0.0364832878112793,
   0.040489912033081055],
  [0.017877817153930664,
   0.05051898956298828,
   0.08045816421508789,
   0.08291888236999512,
   0.08767914772033691,
   0.10440731048583984,
   0.10595130920410156]],
 [0.017351865768432617,
  0.03830512364705404,
  0.04574853181838989,
  0.04685187339782715,
  0.050960969924926755,
  0.05300722122192383,
  0.054484987258911134])

In [19]:
#kako se optimalna vrednost spreminja glede na št. izbrisanih vozlov a
def opt_vrednost_vozli(m,n,max_st_izbrisanih_vozlisc,b,k):
    seznam_vrednosti = []
    for i in range(0,max_st_izbrisanih_vozlisc + 1):
        G = mreza(m,n,i,b)
        stevilo_komponent = connected_components_number(G)
        if stevilo_komponent <= k:
            razdalja = round(najkrajsa_razdalja(G, k)) #round za zaokrževanje števil 
            seznam_vrednosti.append((razdalja))
        else:
            seznam_vrednosti.append(None)
            
    return seznam_vrednosti

#opt_vrednost_vozli(5,5,15,2,3)

# !NEKI PAMETNGA BO TREBA NAREST Z None-i, ker primerjamo jabolka pa hruske pri razdrobljenih grafih pri relativno majhnih k!
#fukncija vrne opt. razdalje R glede na število odstranjenih vozlišč
#FIKSNO:velikost mreze, K, st. izbrisanih povezav
#SPREMINJAM: st_izbrisanih vozlišč
def R_v_odvisnosti_od_spreminjanje_stevila_vozlisc(m,n,max_st_izbrisanih_vozlisc,b,k,stevilo_ponovitev):
    seznam = []
    for i in range(stevilo_ponovitev):
        razdalje = opt_vrednost_vozli(m,n,max_st_izbrisanih_vozlisc,b,k)
        seznam.append(razdalje)

    povprecja = []
    for j in range(max_st_izbrisanih_vozlisc + 1):
        vsota = 0
        stevec = 0
        for i in range(stevilo_ponovitev):
            if seznam[i][j] != None:
                vsota += seznam[i][j]
                stevec += 1
        if stevec == 0:
            povprecja.append(None)
        else:
            povprecja.append(vsota/stevec)
    return seznam, povprecja

R_v_odvisnosti_od_spreminjanje_stevila_vozlisc(5,5,15,2,3,3)

([[3.0,
   3.0,
   2.0,
   3.0,
   3.0,
   3.0,
   3.0,
   3.0,
   None,
   3.0,
   4.0,
   None,
   None,
   None,
   None,
   None],
  [3.0,
   3.0,
   2.0,
   3.0,
   2.0,
   3.0,
   3.0,
   3.0,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None],
  [3.0,
   3.0,
   2.0,
   3.0,
   3.0,
   3.0,
   4.0,
   2.0,
   None,
   5.0,
   None,
   None,
   None,
   None,
   None,
   None]],
 [3.0,
  3.0,
  2.0,
  3.0,
  2.6666666666666665,
  3.0,
  3.3333333333333335,
  2.6666666666666665,
  None,
  4.0,
  4.0,
  None,
  None,
  None,
  None,
  None])

In [21]:
import time

#funkcija drugacna od cas_izvajanja_k(m,n,a,b,k)
#čas izvajanja v odvisnosti od k
def cas_izvajanja_k2(m,n,max_st_izbrisanih_vozlisc,b,k):
    seznam_casov = []
    for i in range(max_st_izbrisanih_vozlisc + 1):
        G = mreza(m,n,i,b)
        stevilo_komponent = connected_components_number(G)
        if stevilo_komponent <= k:
            zacetni = time.time()
            najkrajsa_razdalja(G,k)
            koncni = time.time() - zacetni
            seznam_casov.append(koncni)
        else:
            seznam_casov.append(None)
    return seznam_casov

#cas_izvajanja_k(3,3,1,1,5)

#funkcija vrne povprečne čase (od n ponovitev) glede na spreminjanje vozlov
#FIKSNO:velikost mreze, K, st. izbrisanih povezav
#SPREMINJAM: st_izbrisanih vozlov
def cas_v_odvisnosti_od_spreminjanja_stevila_vozlisc(m,n,max_st_izbrisanih_vozlisc,b,k,stevilo_ponovitev):
    seznam = []
    for i in range(stevilo_ponovitev):
        casi = cas_izvajanja_k2(m,n,max_st_izbrisanih_vozlisc,b,k)
        seznam.append(casi)

    povprecja = []
    for j in range(max_st_izbrisanih_vozlisc + 1):
        vsota = 0
        stevec = 0
        for i in range(stevilo_ponovitev):
            if seznam[i][j] != None:
                vsota += seznam[i][j]
                stevec += 1
        if stevec == 0:
            povprecja.append(None)
        else:
            povprecja.append(vsota/stevec)
    return seznam, povprecja

cas_v_odvisnosti_od_spreminjanja_stevila_vozlisc(5,5,15,2,3,3)

([[5.639472484588623,
   3.921388626098633,
   4.107468843460083,
   2.626314640045166,
   1.0834574699401855,
   0.7895870208740234,
   0.38587093353271484,
   0.7918782234191895,
   None,
   0.18928837776184082,
   0.04175376892089844,
   None,
   None,
   None,
   None,
   None],
  [6.209710121154785,
   4.308660984039307,
   3.489457368850708,
   2.4651668071746826,
   2.166227102279663,
   1.2804343700408936,
   0.4242086410522461,
   0.04644370079040527,
   None,
   0.030963659286499023,
   None,
   None,
   None,
   None,
   None,
   None],
  [4.494164943695068,
   3.5699329376220703,
   2.5669443607330322,
   2.609086275100708,
   1.8911659717559814,
   1.2816765308380127,
   0.15097332000732422,
   0.8892922401428223,
   None,
   0.6349911689758301,
   0.20865154266357422,
   0.035265445709228516,
   None,
   None,
   None,
   None]],
 [5.447782516479492,
  3.9333275159200034,
  3.3879568576812744,
  2.5668559074401855,
  1.7136168479919434,
  1.1172326405843098,
  0.320350964

In [45]:
#FIKSNO: k, st. izbrisanih vozlisc in povezav
#SPREMINJAVA: velikost mreze
#velikost mreže narašča od 3x3 do nxn
def R_v_odvisnosti_od_velikosti_kvadratne_mreze(n,a,b,k,stevilo_ponovitev):
    seznam = []
    for i in range(stevilo_ponovitev):
        seznamcek = []
        for j in range(3, n+1):
            G = mreza(j,j,a,b)
            stevilo_komponent = connected_components_number(G)
            if stevilo_komponent <= k:
                razdalje = round(najkrajsa_razdalja(G, k))
                seznamcek.append(razdalje)
            else:
                seznamcek.append(None)
        seznam.append(seznamcek)

    povprecja = []
    for j in range(0, n-2):
        vsota = 0
        stevec = 0
        for i in range(stevilo_ponovitev):
            if seznam[i][j] != None:
                vsota += seznam[i][j]
                stevec += 1
        if stevec == 0:
            povprecja.append(None)
        else:
            povprecja.append(vsota/stevec)

    return seznam, povprecja


R_v_odvisnosti_od_velikosti_kvadratne_mreze(6,3,2,3,3)


([[1.0, 2.0, 2.0, 3.0], [1.0, 2.0, 2.0, 4.0], [None, 2.0, 2.0, 3.0]],
 [1.0, 2.0, 2.0, 3.3333333333333335])

In [4]:
#FIKSNO: k, st. izbrisanih vozlisc in povezav, ŠT. VRSTIC (m)
#SPREMINJAVA: velikost mreze (število stolpcev: od 3 do n)
#velikost mreže narašča od mx3 do mxn
def R_v_odvisnosti_od_velikosti_mreze(m,n,a,b,k,stevilo_ponovitev):
    seznam = []
    for i in range(stevilo_ponovitev):
        seznamcek = []
        for j in range(3, n+1):
            G = mreza(m,j,a,b)
            stevilo_komponent = connected_components_number(G)
            if stevilo_komponent <= k:
                razdalje = round(najkrajsa_razdalja(G, k))
                seznamcek.append(razdalje)
            else:
                seznamcek.append(None)
        seznam.append(seznamcek)

    povprecja = []
    for j in range(0, n-2):
        vsota = 0
        stevec = 0
        for i in range(stevilo_ponovitev):
            if seznam[i][j] != None:
                vsota += seznam[i][j]
                stevec += 1
        if stevec == 0:
            povprecja.append(None)
        else:
            povprecja.append(vsota/stevec)

    return seznam, povprecja

R_v_odvisnosti_od_velikosti_mreze(4,6,2,3,3,3)

([[2.0, 3.0, 2.0, 3.0], [2.0, 2.0, 2.0, 3.0], [2.0, 2.0, 2.0, 3.0]],
 [2.0, 2.3333333333333335, 2.0, 3.0])

In [18]:
#FIKSNO: st. izbrisanih vozlisc in povezav
#SPREMINJAVA: velikost mreze in maximalno stevilo K-jev
#velikost mreže narašča od 3x3 do nxn
#VRNE matriko: v a_ij pove povprecni R (za n ponovitev) pri mreži ixi, ki ima j centrov
def R_v_odvisnosti_od_velikosti_kvadratne_mreze_in_k(n,a,b,max_k,stevilo_ponovitev):
    seznam = [] # i-ti element tega seznama pove povprecne R-je za razlicne k-je za ixi matriko
    for i in range(3, n+1):
        seznam1 = [] #seznam povprecnega R za stevilo_centrov = j
        for j in range(1, max_k + 1):
            seznam2 = []
            for p in range(stevilo_ponovitev):
                G = mreza(i,i,a,b)
                stevilo_komponent = connected_components_number(G)
                if stevilo_komponent <= j: #st. komponent <= k (st.centrov) (v tem primeru j)
                    razdalja = round(najkrajsa_razdalja(G, j))
                    seznam2.append(razdalja)
                else:
                    seznam2.append(None)

            vsota = 0
            stevec = 0
            povprecje = 0
            for v in range(len(seznam2)):
                if seznam2[v] != None:
                    vsota += seznam2[v]
                    stevec += 1
            if stevec == 0:
                povprecje = None
            else:
                povprecje = vsota/stevec

            seznam1.append(povprecje)

        seznam.append(seznam1)

    return seznam



R_v_odvisnosti_od_velikosti_kvadratne_mreze_in_k(5,3,3,3,3)

[[None, None, 1.0],
 [4.0, 3.5, 2.3333333333333335],
 [5.0, 3.6666666666666665, 3.0]]